In [2]:
import pandas as pd
import numpy as np
def write_submission_file(_model, _featureList): # see submission.ipynb for details
    import pandas as pd
    sample = pd.read_csv('data/sample.csv', index_col=0)
    Xsample = sample[_featureList].values
    sample['Pred'] = _model.predict_proba(Xsample)[:,0]
    submission = sample[['Id', 'Pred']]
    submission.to_csv("data/submission.csv", encoding='ascii')
    
tourney_df = pd.read_csv("data/data.csv", index_col=0)
tourney_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lstl,Lblk,Lpf,Wteam_name,Lteam_name,team0Win,team0,team1,wins0,wins1
0,2003,134,1421,92,1411,84,N,1,32,69,...,5,0,22,UNC Asheville,TX Southern,0,1411,1421,18,13
1,2003,136,1112,80,1436,51,N,0,31,66,...,10,3,15,Arizona,Vermont,1,1112,1436,25,19
2,2003,136,1113,84,1272,71,N,0,31,59,...,2,5,18,Arizona St,Memphis,1,1113,1272,18,23
3,2003,136,1141,79,1166,73,N,0,29,53,...,6,6,21,C Michigan,Creighton,0,1166,1141,29,23
4,2003,136,1143,76,1301,74,N,1,27,64,...,5,8,19,California,NC State,1,1143,1301,21,18


# Choosing Features to Use for Model

Edit featureList below to use any of the features in the data frame above (see above for names of columns).  

In [3]:
featureList = ['wins0', 'wins1']
y = tourney_df['team0Win'].values # results
X = tourney_df[featureList].values # features

# Train our model!

In [4]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:42: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
from sklearn import svm
model = svm.SVC(probability=True)
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
y_pred = model.predict_proba(X_test) # probability that team0 wins (what Kaggle calls team 1, and wants for submission)
for i in range(5):
    print("Features = {0}, Pred. prob. team0 wins: {1}".format(X_test[i], y_pred[i]))

Features = [26 30], Pred. prob. team0 wins: [ 0.49268495  0.50731505]
Features = [33 25], Pred. prob. team0 wins: [ 0.42665838  0.57334162]
Features = [27 32], Pred. prob. team0 wins: [ 0.5  0.5]
Features = [22 24], Pred. prob. team0 wins: [ 0.38112727  0.61887273]
Features = [29 26], Pred. prob. team0 wins: [ 0.5  0.5]


# Calculating logloss 

Now the target vector is modified to be team0Win, when team0Win, y=1, which is consistant with the Koggle definition and the github definition. Please compare http://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html and https://www.kaggle.com/c/march-machine-learning-mania-2016/details/evaluation to make sure we're doing the same thing.

In [8]:
from sklearn import metrics
print("Log loss is {0}".format(metrics.log_loss(y_test, y_pred)))

Log loss is 0.649180121261


# Making a submission on Kaggle

The following call generates a file `data/submission.csv` that uses the model you pass it to generate probabilities for all the matches Kaggle wants us to predict.  It will overwrite any previous file.  See `submission.ipynb` for a walk-through of the function.  Have to pass it the model you trained, and the featureList you used.

In [8]:
write_submission_file(model, featureList)